In [ ]:
import pandas as pd
import numpy as np
import datetime as dt

def make_datetime(x):
    # string 타입의 Time column을 datetime 타입으로 변경
    x     = str(x)
    year  = int(x[:4])
    month = int(x[4:6])
    day   = int(x[6:8])
    hour  = int(x[8:10])
    minute  = int(x[10:12])
    sec  = int(x[12:])
    return dt.datetime(year, month, day, hour,minute)

def string2num(x):
    # (,)( )과 같은 불필요한 데이터 정제
    x = re.sub(r"[^0-9]+", '', str(x))
    if x =='':
        return 0
    else:
        return int(x)
    
def string2num2(x):
    # (,)( )과 같은 불필요한 데이터 정제
    x = re.sub(r"[^0-9]+", '', str(x))
    return int(x)

PATH = "Jupyter Temp/lg 공모전/data/"

In [2]:
pd.set_option("display.max_rows",None)
pd.set_option("display.max_columns",None)

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
train_err  = pd.read_csv(PATH+'train_err_data.csv')
train_qual  = pd.read_csv(PATH+'train_quality_data.csv')
train_prob  = pd.read_csv(PATH+'train_problem_data.csv')

C:\Users\sue_b\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (10,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [9]:
model_train = pd.DataFrame(data={"user_id": train_err["user_id"].unique()})

In [11]:
model_train.sort_values(by="user_id",inplace=True)
model_train.head()

,user_id
0,10000
1,10001
2,10002
3,10003
4,10004


In [17]:
model_train = model_train.merge(train_err.groupby(by="user_id")["model_nm"].nunique().reset_index(),how="left",on="user_id")
model_train.head()

,user_id,model_nm
0,10000,1
1,10001,1
2,10002,1
3,10003,1
4,10004,1


In [18]:
train_err.head()

,user_id,time,model_nm,fwver,errtype,errcode
0,10000,20201101025616,model_3,05.15.2138,15,1
1,10000,20201101030309,model_3,05.15.2138,12,1
2,10000,20201101030309,model_3,05.15.2138,11,1
3,10000,20201101050514,model_3,05.15.2138,16,1
4,10000,20201101050515,model_3,05.15.2138,4,0


In [20]:
def model6(x):
    if x == "8.5.3":
        result = "10.08.53"
    elif x == "10":
        result = "10.00.00"
    else:
        result = x
    return result

In [22]:
train_err["fwver"] = train_err["fwver"].apply(lambda x : model6(x))

In [24]:
train_err["fwver"] = train_err["fwver"].apply(lambda x : x[-4:])

In [25]:
train_err.head()

,user_id,time,model_nm,fwver,errtype,errcode
0,10000,20201101025616,model_3,2138,15,1
1,10000,20201101030309,model_3,2138,12,1
2,10000,20201101030309,model_3,2138,11,1
3,10000,20201101050514,model_3,2138,16,1
4,10000,20201101050515,model_3,2138,4,0


In [31]:
temp = train_err.groupby(by=["user_id","model_nm"])["fwver"].nunique().reset_index()

In [34]:
temp["update"] = temp["fwver"].apply(lambda x : x-1)

In [37]:
temp = temp.groupby(by="user_id")["update"].sum().reset_index()

In [38]:
temp.head()

,user_id,update
0,10000,0
1,10001,1
2,10002,0
3,10003,1
4,10004,1


In [39]:
model_train = model_train.merge(temp,how="left",on="user_id")

In [40]:
model_train.head()

,user_id,model_nm,update
0,10000,1,0
1,10001,1,1
2,10002,1,0
3,10003,1,1
4,10004,1,1


In [44]:
model_train.to_csv("Jupyter Temp/lg 공모전/data/0124_train.csv",index=False)

In [47]:
temp = train_err[["user_id","errcode"]].copy()

In [49]:
pd.get_dummies(temp,prefix="errcode")

MemoryError: Unable to allocate 43.2 GiB for an array with shape (16554663, 2805) and data type uint8